In [1]:
import pandas as pd
import re
import nltk

from nltk.corpus import sentiwordnet as swn

In [9]:
정상기업뉴스 = pd.read_csv('정상기업번역파일.csv')
정상기업뉴스.drop('뉴스기사본문전처리', axis=1, inplace=True)

In [3]:
a = [re.contains('[가-힣]', s) for s in 정상기업뉴스['기사본문번역']]
a

AttributeError: module 're' has no attribute 'contains'

In [4]:
상폐기업뉴스[상폐기업뉴스['기사본문번역'].str.contains('[가-힣]')]['기사본문번역']

294     미국 신용등급 강등인한 글로벌 증시 폭락으로 인해 국내 증시도 한치 앞을 내다 볼 ...
480     Looking at the appearance of the stock market ...
481     Looking at the appearance of the stock market ...
482     상승과 하락을 반복하면서 변동성이 확대되는 주식시장 흐름에서는 손실을 줄이고 차익실...
539     Ko Young Technology announced on the 1st that ...
655     Laptop monitor after light emitting diode disp...
1076    Eugene companies with quarterly sales of KRW 1...
1498    Reporter Park Cho-rong's Leadcorp announced in...
1667    Reporter Shin Jae-woo, EM Korea, announced tha...
1683    Reporter Lee Sang-hyun, the Korea Hydrogen Ind...
2069    Meaning that China store Clio Clio tint cosmet...
2087    Among related stocks, TES, which has recorded ...
2210    서울뉴시스 한국거래소 코스닥시장본부는 일 오전 서울 여의도 한국거래소 서울사옥 홍보...
Name: 기사본문번역, dtype: object

In [ ]:
정상기업뉴스['기사본문번역토큰'] = 정상기업뉴스['기사본문번역'].apply(lambda x: nltk.word_tokenize(x))
정상기업뉴스['기사본문번역토큰'] = 정상기업뉴스['기사본문번역토큰'].apply(lambda x: nltk.pos_tag(x))

In [10]:
def word_sentiment_calculator(word, tag):
    pos_score = 0
    neg_score = 0
    
    if 'NN' in tag and len(list(swn.senti_synsets(word, 'n')))>0:
        syn_set = list(swn.senti_synsets(word, 'n'))
    elif 'VB' in tag and len(list(swn.senti_synsets(word, 'v')))>0:
        syn_set = list(swn.senti_synsets(word, 'v'))
    elif 'JJ' in tag and len(list(swn.senti_synsets(word, 'a')))>0:
        syn_set = list(swn.senti_synsets(word, 'a'))
    elif 'RB' in tag and len(list(swn.senti_synsets(word, 'r')))>0:
        syn_set = list(swn.senti_synsets(word, 'r'))
    else:
        return (0,0)
    
    for syn in syn_set:
        pos_score += syn.pos_score()
        neg_score += syn.neg_score()
    return (pos_score/len(syn_set), neg_score/len(syn_set))

In [11]:
def sentence_sentiment_calculator(sent):
    tokens =  nltk.word_tokenize(sent)
    pos_tags = nltk.pos_tag(tokens)
    
    pos_score = 0
    neg_score = 0
    for word, tag in pos_tags:
        pos_score += word_sentiment_calculator(word, tag)[0]
        neg_score += word_sentiment_calculator(word, tag)[1]
    return (pos_score, neg_score)

In [12]:
## 긍정일경우 0 부정일경우 1
predicted = []

for i in range(len(정상기업뉴스)):
    scores = sentence_sentiment_calculator(정상기업뉴스.기사본문번역[i])
    
    if scores[0] >= scores[1]:
        predicted.append(0)
    else:
        predicted.append(1)

정상기업뉴스['긍정부정분류'] = predicted

In [13]:
기업 = 정상기업뉴스.groupby('기업').count()['긍정부정분류'].index
뉴스 = 정상기업뉴스.groupby('기업').count()['긍정부정분류'].values
전체기사수 = pd.DataFrame(columns=['기업명', '기사수'])
전체기사수['기업명'] = 기업
전체기사수['기사수'] = 뉴스

부정기사기업 = 정상기업뉴스[정상기업뉴스.긍정부정분류==1].groupby('기업').count()['긍정부정분류'].index
부정기사뉴스 = 정상기업뉴스[정상기업뉴스.긍정부정분류==1].groupby('기업').count()['긍정부정분류'].values
부정기사수 = pd.DataFrame(columns=['기업명', '부정기사수'])
부정기사수['기업명'] = 부정기사기업
부정기사수['부정기사수'] = 부정기사뉴스

정상기업 = pd.merge(전체기사수, 부정기사수, on=['기업명'], how='left')
정상기업['부정기사비율'] = 정상기업['부정기사수'] / 정상기업['기사수']
정상기업

,기업명,기사수,부정기사수,부정기사비율
0,경창산업,15,1.0,0.066667
1,고영테크놀러지,25,1.0,0.040000
2,광림,11,2.0,0.181818
3,금화피에스시,6,1.0,0.166667
4,나스미디어,25,2.0,0.080000
...,...,...,...,...
103,하이록코리아,31,1.0,0.032258
104,한국정보통신,7,1.0,0.142857
105,해성산업,7,1.0,0.142857
106,홈캐스트,7,NaN,NaN


In [81]:
상폐기업[상폐기업.기사수 >10].describe()

,기사수,부정기사수,부정기사비율
count,105.000000,105.000000,105.000000
mean,24.676190,8.047619,0.327686
std,8.185935,4.234431,0.126682
min,11.000000,1.000000,0.076923
25%,19.000000,5.000000,0.250000
50%,25.000000,7.000000,0.320000
75%,29.000000,10.000000,0.388889
max,57.000000,23.000000,0.703704


In [16]:
정상기업[정상기업['부정기사비율'].isnull()]

,기업명,기사수,부정기사수,부정기사비율
12,덱스터스튜디오,5,NaN,NaN
15,동원개발,26,NaN,NaN
16,동진쎄미켐,5,NaN,NaN
18,디티앤씨,17,NaN,NaN
23,마크로젠,23,NaN,NaN
24,멀티캠퍼스,13,NaN,NaN
29,블루콤,37,NaN,NaN
31,상상인,2,NaN,NaN
32,서부티엔디,2,NaN,NaN
33,서원인텍,3,NaN,NaN


In [50]:
부정기사수 = 정상기업뉴스[정상기업뉴스.긍정부정분류==1].groupby('기업').count()['긍정부정분류'].values

In [32]:
data1 = pd.read_csv('../재무비율모델/Dataset1_재무비율.csv')
data1 = data1[data1.부실기업여부==1]
data1

,회사명,회계년도,운전자산총자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율,폐지일자,부실기업여부
0,네이쳐글로벌,2009,74.492049,-49.803589,-3.371182,673.432622,6.028268,2011-01-08,1
1,네프로아이티,2009,-0.465154,19.338150,13.884726,157.344363,76.780711,2011-11-05,1
2,뉴젠아이씨티,2009,27.593677,-1075.333708,-28.342808,136.342262,40.747316,2011-04-29,1
3,트루아워,2009,52.368738,-19.102365,-25.573719,412.297157,33.436299,2011-10-25,1
4,디패션,2009,20.920391,-333.294194,-23.158301,170.052742,71.074703,2011-05-07,1
...,...,...,...,...,...,...,...,...,...
130,위너지스,2016,4.206434,-376.016367,-21.360033,105.778735,78.213923,2018-10-11,1
131,위노바,2016,28.671608,-89.404580,-9.113157,487.272885,63.716291,2018-03-09,1
132,트레이스,2016,3.857923,-35.621815,0.788938,238.537763,22.490823,2018-10-11,1
133,레이젠,2016,-3.874133,-5.069385,-9.353084,190.706311,117.325339,2018-10-11,1
